[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gbatt55/ai-ontology-map/blob/main/notebooks/visualize_graph.ipynb)

# AI Supply Chain Ontology — Graph Visualizer

This notebook visualizes the ontology graph defined in `ontology.json`.

Features:
- Auto-detects `ontology.json` whether running in GitHub, Colab, or locally.
- Visualizes selected layers (single or multiple).
- Uses NetworkX + Matplotlib.

Edit the `selected_layers` list below to control which layers are shown.


In [ ]:
# Install networkx (safe to run multiple times)
!pip install networkx
import networkx as nx
import matplotlib.pyplot as plt
import os, json

In [ ]:
# === AUTO-DETECT ONTOLOGY.JSON PATH ===
possible_paths = [
    'ontology.json',
    '../ontology.json',
    '/content/ontology.json',
    '/content/drive/MyDrive/ontology.json'
]

ONTOLOGY_PATH = None

for p in possible_paths:
    if os.path.exists(p):
        ONTOLOGY_PATH = p
        break

if ONTOLOGY_PATH is None:
    raise FileNotFoundError('ontology.json not found in any known path.')

print(f'Using ontology path: {ONTOLOGY_PATH}')

with open(ONTOLOGY_PATH, 'r') as f:
    data = json.load(f)

nodes = data['nodes']
edges = data['edges']

print(f'Loaded {len(nodes)} nodes and {len(edges)} edges.')

In [ ]:
# === CONFIG — SELECT LAYERS TO VISUALIZE ===
# Example:
#   [2] → only Layer 2
#   [1,2,3] → Layers 1–3
#   [] → ALL layers

selected_layers = [2]  # <-- edit this line

# Node size
NODE_SIZE = 900

# === BUILD GRAPH ===
G = nx.DiGraph()

for n in nodes:
    G.add_node(n['id'], **n)

for e in edges:
    G.add_edge(e['source'], e['target'], **e)

# === FILTER ===
if selected_layers:
    H = nx.DiGraph()
    for n, d in G.nodes(data=True):
        if d.get('layer') in selected_layers:
            H.add_node(n, **d)
    for u, v, d in G.edges(data=True):
        if u in H.nodes and v in H.nodes:
            H.add_edge(u, v, **d)
else:
    H = G.copy()

if H.number_of_nodes() == 0:
    print('No nodes to display for selected_layers =', selected_layers)
else:
    cmap = plt.cm.get_cmap('tab10')
    def color_for_layer(layer):
        if layer is None: return (0.5, 0.5, 0.5, 1)
        return cmap((layer - 1) % 10)

    node_colors = [color_for_layer(d.get('layer')) for n, d in H.nodes(data=True)]
    labels = {n: d.get('name', n) for n, d in H.nodes(data=True)}

    pos = nx.spring_layout(H, seed=42, k=1.1)

    plt.figure(figsize=(14, 9))

    nx.draw_networkx_nodes(H, pos,
                           node_color=node_colors,
                           node_size=NODE_SIZE,
                           alpha=0.9)

    dep = [(u, v) for u, v, d in H.edges(data=True) if d.get('type') == 'dependency']
    comp = [(u, v) for u, v, d in H.edges(data=True) if d.get('type') == 'competition']
    lev = [(u, v) for u, v, d in H.edges(data=True) if d.get('type') == 'leverage']

    nx.draw_networkx_edges(H, pos, edgelist=dep, arrows=True, arrowstyle='-|>', alpha=0.7)
    nx.draw_networkx_edges(H, pos, edgelist=lev, arrows=True, arrowstyle='-|>', style='dashed', alpha=0.7)
    nx.draw_networkx_edges(H, pos, edgelist=comp, arrows=False, style='dotted', alpha=0.7)

    nx.draw_networkx_labels(H, pos, labels, font_size=9)

    title = 'AI Supply Chain Ontology — Layers ' + (str(selected_layers) if selected_layers else 'ALL')
    plt.title(title)
    plt.axis('off')
    plt.show()

    present = sorted({d.get('layer') for _, d in H.nodes(data=True)})
    print('Layers present in this view:', present)
